In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

'''
training_data = pd.read_csv('C:/Users/is_li/Desktop/paper/github/stock on MLOps/MLOps/tools/DVC/example/data/train.csv',
index_col= [0,1],header=[0,1])
valid_data = pd.read_csv('C:/Users/is_li/Desktop/paper/github/stock on MLOps/MLOps/tools/DVC/example/data/valid.csv',
index_col= [0,1],header=[0,1])
test_data = pd.read_csv('C:/Users/is_li/Desktop/paper/github/stock on MLOps/MLOps/tools/DVC/example/data/test.csv',
index_col= [0,1],header=[0,1])
'''
df_data  = pd.read_csv('C:/Users/is_li/Desktop/paper/github/stock on MLOps/MLOps/tools/DVC/example/data/dataset.csv',
index_col= [0,1],header=[0,1])
df_data = df_data.fillna(0)

c:\ProgramData\Anaconda3\envs\py380\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional	
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [3]:

# 创建两个列表，用来存储数据的特征和标签
data_feat, data_target = [],[]

# 设每条数据序列有20组数据
seq = 10
feature_num = 10

for index in range(len(df_data) - seq):
    # 构建特征集
    data_feat.append(df_data['feature'].iloc[:,range(feature_num)][index: index + seq].values)
    # 构建target集
    data_target.append(df_data['label'][index:index + seq])

# 将特征集和标签集整理成numpy数组
data_feat = np.array(data_feat)
data_target = np.array(data_target)


In [4]:
# 这里按照8:2的比例划分训练集和测试集
test_set_size = int(np.round(0.1*df_data.shape[0]))  # np.round(1)是四舍五入，
train_size = data_feat.shape[0] - (test_set_size) 
print(test_set_size)  # 输出测试集大小
print(train_size)     # 输出训练集大小

611
5489


In [5]:

# 这里第一个维度自动确定，我们认为其为batch_size，因为在LSTM类的定义中，设置了batch_first=True
trainX = torch.from_numpy(data_feat[:train_size].reshape(-1,seq,feature_num)).type(torch.Tensor)   
testX  = torch.from_numpy(data_feat[train_size:].reshape(-1,seq,feature_num)).type(torch.Tensor)
trainY = torch.from_numpy(data_target[:train_size].reshape(-1,seq,1)).type(torch.Tensor)
testY  = torch.from_numpy(data_target[train_size:].reshape(-1,seq,1)).type(torch.Tensor)
print('x_train.shape = ',trainX.shape)
print('y_train.shape = ',trainY.shape)
print('x_test.shape = ',testX.shape)
print('y_test.shape = ',testY.shape)



x_train.shape =  torch.Size([5489, 10, 10])
y_train.shape =  torch.Size([5489, 10, 1])
x_test.shape =  torch.Size([611, 10, 10])
y_test.shape =  torch.Size([611, 10, 1])


In [6]:
batch_size=5
train = torch.utils.data.TensorDataset(trainX,trainY)
test = torch.utils.data.TensorDataset(testX,testY)
train_loader = torch.utils.data.DataLoader(dataset=train, 
                                           batch_size=batch_size, 
                                           shuffle=False)

test_loader = torch.utils.data.DataLoader(dataset=test, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [7]:
import torch.nn as nn

input_dim = feature_num    # 数据的特征数
hidden_dim = 2    # 隐藏层的神经元个数
num_layers = 2     # LSTM的层数
output_dim = 1     # 预测值的特征数
                   #（这是预测股票价格，所以这里特征数是1，如果预测一个单词，那么这里是one-hot向量的编码长度）
class LSTM(nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size=1, num_layers=2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers)
        self.fc = nn.Linear(hidden_size, output_size)
    def forward(self, _x):
        x, _ = self.lstm(_x)  # _x is input, size (seq_len, batch, input_size)
        s, b, h = x.shape  # x is output, size (seq_len, batch, hidden_size)
        x = x.view(s * b, h)
        x = self.fc(x)
        x = x.view(s, b, -1)  # 把形状改回来
        return x

import mlflow
import mlflow.pytorch
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder

# 设置MLflow追踪URI
mlflow.set_tracking_uri("file:/path/to/mlflow")

# 设置实验名称
mlflow.set_experiment("stock-price-prediction")

# 定义数据集类
class StockDataset(Dataset):
    def __init__(self, data):
        self.data = data
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]

# 定义LSTM模型类
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x shape: (seq_len, batch, input_size)
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

# 定义训练函数
def train_model(model, train_loader, test_loader, loss_func, optimizer, device, epochs):
    for epoch in range(epochs):
        # 训练模式
        model.train()
        train_loss = 0.0
        for i, (feat, target) in enumerate(train_loader):
            feat, target = feat.to(device), target.to(device)
            optimizer.zero_grad()
            pred = model(feat)
            loss = loss_func(pred, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)

        # 测试模式
        model.eval()
        test_loss = 0.0
        with torch.no_grad():
            for feat, target in test_loader:
                feat, target = feat.to(device), target.to(device)
                pred = model(feat)
                loss = loss_func(pred, target)
                test_loss += loss.item()
            test_loss /= len(test_loader)

        # 记录日志
        mlflow.log_metric("train_loss", train_loss, epoch)
        mlflow.log_metric("test_loss", test_loss, epoch)
        print(f"Epoch {epoch+1}, Train Loss: {train_loss:.6f}, Test Loss: {test_loss:.6f}")

# 读取数据集
df_data = pd.read_csv('C:/Users/is_li/Desktop/paper/github/stock on MLOps/MLOps/tools/DVC/example/data/dataset.csv', index_col=[0,1], header=[0,1])
df_data = df_data.fillna(0)

# 特征编码
label_encoder = LabelEncoder()
df_data[('feature', 'stock')] = label_encoder.fit_transform(df_data[('feature', 'stock')])

# 构建数据集
data = []
seq_len = 10
for i in range(len(df_data)-seq_len):
    feat = df_data[('feature', slice(None))].iloc[i:i+seq_len].values
    target = df_data['label'].iloc[i+seq_len]
    data.append((feat, target))

train_size = int(len(data)*0.8)
train_set = StockDataset(data[:train_size])
test_set = StockDataset(data[train_size:])
train


In [ ]:
import mlflow
import mlflow
import torch
import torch.nn as nn
import torch.optim as optim
# Start an MLflow experiment
mlflow.set_experiment("Model Training")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


def train_model(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(dataloader, 0):
        # 将数据移动到设备上
        inputs, labels = inputs.to(device), labels.to(device)

        # 清零梯度
        optimizer.zero_grad()

        # 前向传播
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # 反向传播和优化
        loss.backward()
        optimizer.step()

        # 统计损失
        running_loss += loss.item()

    # 返回平均损失
    return running_loss / len(dataloader)


def evaluate_model(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloader, 0):
            # 将数据移动到设备上
            inputs, labels = inputs.to(device), labels.to(device)

            # 前向传播
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # 统计损失
            running_loss += loss.item()

            # 统计正确率
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # 返回平均损失和准确率
    return running_loss / len(dataloader), correct / total


# Log the parameters and metrics for each run
with mlflow.start_run():
    mlflow.log_param("lr", 0.01)
    mlflow.log_param("batch_size", 32)
    mlflow.log_param("dropout", 0.2)
    
    model = train_model(lr=0.01, batch_size=32, dropout=0.2)
    loss, accuracy = evaluate_model(model)
    
    mlflow.log_metric("loss", loss)
    mlflow.log_metric("accuracy", accuracy)
    
    # Save the model as an artifact
    mlflow.pytorch.log_model(model, "models")

num_epochs = 50
loss_function = nn.MSELoss()  # 损失函数
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # 优化器

# 定义 MLflow 参数
experiment_name = "stock-price-prediction"
run_name = "LSTM"
params = {
    "num_epochs": num_epochs,
    "input_dim": input_dim,
    "hidden_dim": hidden_dim,
    "num_layers": num_layers,
    "output_dim": output_dim,
    "batch_size": batch_size,
    "learning_rate": 0.1
}

# 启动 MLflow 实验并记录参数
mlflow.set_experiment(experiment_name)
with mlflow.start_run(run_name=run_name):
    # 记录参数
    mlflow.log_params(params)

    # 训练模型
    train_loss = []
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            pred = model(batch_X)
            loss = loss_function(pred, batch_y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        train_loss.append(epoch_loss / len(train_loader))

        # 每10个 epoch 记录一次
        if (epoch + 1) % 10 == 0:
            test_loss = evaluate_model(model, test_loader, loss_function)
            print('Epoch: {}, Train Loss: {:.10f}, Test Loss: {:.10f}'.format(epoch + 1, train_loss[-1], test_loss))
            mlflow.log_metric("train_loss", train_loss[-1], step=epoch)
            mlflow.log_metric("test_loss", test_loss, step=epoch)

    # 在 MLflow 中记录模型和参数
    mlflow.pytorch.log_model(model, "models")
    mlflow.log_artifact("stock-price-prediction.py")



In [9]:

# 定义模型
num_epochs = 50
model = LSTM(input_size=input_dim, hidden_size=hidden_dim, output_size=output_dim, num_layers=num_layers)
# print(model)
# 打印模型各层的参数尺寸

for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())
    
loss_function = nn.MSELoss()  # 损失函数
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # 优化器



train_loss = [] 
for epoch in range(num_epochs):
    out = model(trainX)
    loss = loss_function(out, trainY)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    train_loss.append(loss.item())
    if (epoch + 1) % 10 == 0:
        print('Epoch: {}, Loss:{:.10f}'.format(epoch + 1, loss.item()))




c:\ProgramData\Anaconda3\envs\py380\lib\site-packages\nni\runtime\platform\standalone.py:32: RuntimeWarning: Running trial code without runtime. Please check the tutorial if you are new to NNI: https://nni.readthedocs.io/en/stable/tutorials/hpo_quickstart_pytorch/main.html
  warnings.warn(warning_message, RuntimeWarning)


KeyError: 'hidden_dim'

In [ ]:
# 输出NNI最终结果
nni.report_final_result(loss.item())

In [ ]:
# 绘制loss函数图像
plt.plot(train_loss)
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()